In [73]:
import pandas as pd
import datetime
import numpy as np

from IPython.display import display


In [66]:
logfile_names = ["./data/all_jobs_202103.csv", "./data/all_jobs_202108.csv", "./data/all_jobs_202111.csv" ]
for logfile_name in logfile_names:
    data = pd.read_csv(logfile_name)
    data.sort_values("Submit")

    num_submit = data["Submit"].count()
    first_submit = datetime.datetime.fromtimestamp(data["Submit"][0])
    last_submit = datetime.datetime.fromtimestamp(data["Submit"][data["Submit"].size-1])
    print(f"{num_submit:,} jobs from {first_submit} to {last_submit}")


68,552 jobs from 2021-03-04 00:05:41 to 2021-04-03 23:33:35
171,022 jobs from 2021-08-06 00:12:29 to 2021-09-05 23:58:26
290,967 jobs from 2021-11-07 00:12:52 to 2021-11-30 23:59:57


In [77]:
data = pd.read_csv("./data/all_jobs_2021.csv")
data

,JobName,ReqMem,ReqCPUs,ReqGPUs,ReqDuration,ActualDuration,Submit
0,job1,20000000,32,2,2160000,4,1607433935
1,job2,16000000,1,0,2163600,2,1607433935
2,job3,16000000,1,0,2163600,3,1607433935
3,job4,20000000,6,1,2160000,3,1607433935
4,job5,20000000,32,2,2160000,4,1607433935
...,...,...,...,...,...,...,...
1474800,job1474801,12000000,1,0,345600,7,1638338377
1474801,job1474802,12000000,1,0,345600,26,1638338378
1474802,job1474803,12000000,1,0,345600,11,1638338391
1474803,job1474804,12000000,1,0,345600,13,1638338391


In [98]:
# Go week by week and see how many jobs were submitted by week
overall_end = datetime.datetime(2022, 1, 1)

period_start = datetime.datetime(2021, 1, 1)

one_week = datetime.timedelta(days=7)
one_day = datetime.timedelta(days=1)

period_end = period_start + one_week
start_timestamp = period_start.timestamp()
end_timestamp = period_end.timestamp()

job_quantities = []

while period_end < overall_end:
    hits = np.where((data.Submit >= start_timestamp) & (data.Submit < end_timestamp))
    this_weeks_jobs = data.loc[hits]
    if this_weeks_jobs.Submit.count() == 0:
        period_start = period_end
        period_end = period_start + one_week
        start_timestamp = period_start.timestamp()
        end_timestamp = period_end.timestamp()
        continue
    
    print(f"From {period_start} to {period_end}: {this_weeks_jobs.Submit.count()} jobs")
    job_quantities.append( (period_start, period_end, this_weeks_jobs.Submit.count() ) )
    period_start = period_end
    period_end = period_start + one_week
    start_timestamp = period_start.timestamp()
    end_timestamp = period_end.timestamp()


From 2021-01-01 00:00:00 to 2021-01-08 00:00:00: 10785 jobs
From 2021-01-08 00:00:00 to 2021-01-15 00:00:00: 37587 jobs
From 2021-01-15 00:00:00 to 2021-01-22 00:00:00: 29455 jobs
From 2021-01-22 00:00:00 to 2021-01-29 00:00:00: 29054 jobs
From 2021-01-29 00:00:00 to 2021-02-05 00:00:00: 38588 jobs
From 2021-02-05 00:00:00 to 2021-02-12 00:00:00: 33454 jobs
From 2021-02-12 00:00:00 to 2021-02-19 00:00:00: 24984 jobs
From 2021-02-19 00:00:00 to 2021-02-26 00:00:00: 19117 jobs
From 2021-02-26 00:00:00 to 2021-03-05 00:00:00: 38529 jobs
From 2021-03-05 00:00:00 to 2021-03-12 00:00:00: 18349 jobs
From 2021-03-12 00:00:00 to 2021-03-19 00:00:00: 33147 jobs
From 2021-03-19 00:00:00 to 2021-03-26 00:00:00: 5267 jobs
From 2021-03-26 00:00:00 to 2021-04-02 00:00:00: 8331 jobs
From 2021-04-02 00:00:00 to 2021-04-09 00:00:00: 11500 jobs
From 2021-04-09 00:00:00 to 2021-04-16 00:00:00: 11238 jobs
From 2021-04-16 00:00:00 to 2021-04-23 00:00:00: 13289 jobs
From 2021-04-23 00:00:00 to 2021-04-30 00:

In [102]:
groups_to_keep = [5727, 13198, 28072]
low_med_high = []
for quantity in groups_to_keep:
    index = [num_jobs for start, end, num_jobs in job_quantities].index(quantity)
    print(job_quantities[index])
    low_med_high.append(job_quantities[index])

(datetime.datetime(2021, 5, 7, 0, 0), datetime.datetime(2021, 5, 14, 0, 0), 5727)
(datetime.datetime(2021, 6, 18, 0, 0), datetime.datetime(2021, 6, 25, 0, 0), 13198)
(datetime.datetime(2021, 8, 20, 0, 0), datetime.datetime(2021, 8, 27, 0, 0), 28072)


In [110]:
hits = np.where((data.Submit >= low_med_high[0][0].timestamp()) & (data.Submit < low_med_high[0][1].timestamp()))
low = data.loc[hits]

hits = np.where((data.Submit >= low_med_high[1][0].timestamp()) & (data.Submit < low_med_high[1][1].timestamp()))
med = data.loc[hits]

hits = np.where((data.Submit >= low_med_high[2][0].timestamp()) & (data.Submit < low_med_high[2][1].timestamp()))
high = data.loc[hits]

low.to_csv("./data/low_util.csv", index=False)
med.to_csv("./data/med_util.csv", index=False)
high.to_csv("./data/high_util.csv", index=False)